In [1]:
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from torch.utils.data import TensorDataset, DataLoader

# 1. Cargar datos
df = pd.read_csv(r'C:\Users\User win10pro\Documents\herramientas\herramientas-2025\aaa_2\BMW.csv')

# 2. Codificar todo
df_encoded = df.copy()
for col in df_encoded.select_dtypes(include=['object']).columns:
    df_encoded[col] = LabelEncoder().fit_transform(df_encoded[col].astype(str))

# 3. X e y
X = df_encoded.drop(columns=['Sales_Classification', 'Price_USD'])
y = df_encoded['Sales_Classification']

# 4. División
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# 5. A PyTorch
X_train = torch.tensor(X_train.values, dtype=torch.float32)
X_test = torch.tensor(X_test.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.long)
y_test = torch.tensor(y_test.values, dtype=torch.long)

train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# 6. Red neuronal
class Classifier(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(32, 2)  # 2 clases: High / Low
        )

    def forward(self, x):
        return self.net(x)

model = Classifier(X_train.shape[1])
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 7. Entrenamiento
epochs = 100
for epoch in range(epochs):
    model.train()
    for xb, yb in train_loader:
        pred = model(xb)
        loss = criterion(pred, yb)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

# 8. Evaluación
model.eval()
with torch.no_grad():
    preds = model(X_test).argmax(dim=1)
    print(classification_report(y_test, preds, target_names=['Low', 'High']))
    # Fila codificada (según LabelEncoder usado)
    torch.save(model.state_dict(), "bmw_classifier.pth")
print("Modelo guardado como bmw_classifier.pth")

Epoch 0, Loss: 1.2489
Epoch 10, Loss: 0.5680
Epoch 20, Loss: 0.4741
Epoch 30, Loss: 0.0077
Epoch 40, Loss: 0.0614
Epoch 50, Loss: 0.0605
Epoch 60, Loss: 0.2858
Epoch 70, Loss: 0.0597
Epoch 80, Loss: 0.0318
Epoch 90, Loss: 0.1364
              precision    recall  f1-score   support

         Low       0.99      0.99      0.99      3032
        High       1.00      1.00      1.00      6968

    accuracy                           0.99     10000
   macro avg       0.99      0.99      0.99     10000
weighted avg       0.99      0.99      0.99     10000

Modelo guardado como bmw_classifier.pth


In [4]:
import torch
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Recrea el modelo (misma arquitectura)
model = Classifier(X_train.shape[1])
model.load_state_dict(torch.load("bmw_classifier.pth"))
model.eval()

# Codificadores (debes guardarlos o recrearlos)
# Aquí asumes que ya tienes los LabelEncoder iguales

# Ejemplo: predice una fila nueva
nueva_fila = { 'Model': 5,  # X1
    'Year': 14,  # 2024
    'Region': 0,  # Asia
    'Color': 4,  # Grey
    'Fuel_Type': 1,  # Electric
    'Transmission': 0,  # Automatic
    'Engine_Size_L': 2.1,
    'Mileage_KM': 5000,
    'Sales_Volume': 1200
}